In [1]:
import os
import torch
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# Constants
BATCH = 16
IM_SIZE = 224
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load the data
df = pd.read_csv("train.csv")

# Filter dataset with hotel_ids >= 20
s1 = df["hotel_id"].value_counts()
s2 = s1[s1 >= 20]
hotel_id_list = s2.index.tolist()
df1 = df[df["hotel_id"].isin(hotel_id_list)]

# Train-test split (90:10)
train_df, test_df = train_test_split(df1, test_size=0.1, stratify=df1['hotel_id'], random_state=42)

# Map hotel_id to indices for classification
unique_hotel_ids = sorted(train_df['hotel_id'].unique())
hotel_id_to_index = {hotel_id: idx for idx, hotel_id in enumerate(unique_hotel_ids)}
index_to_hotel_id = {idx: hotel_id for hotel_id, idx in hotel_id_to_index.items()}

# Add the class index to the DataFrame
train_df['class_index'] = train_df['hotel_id'].map(hotel_id_to_index)
test_df['class_index'] = test_df['hotel_id'].map(hotel_id_to_index)

# Define transformations
Transform = transforms.Compose([
    transforms.Resize((IM_SIZE, IM_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

# Dataset Class
class HotelImageDataset(Dataset):
    def __init__(self, df, image_dir, transform=None):
        self.df = df
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name = self.df.iloc[idx]['image']
        label = self.df.iloc[idx]['class_index']
        img_path = os.path.join(self.image_dir, str(self.df.iloc[idx]['chain']), img_name)
        
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        
        return image, label

# Initialize datasets and loaders
image_dir = r"C:\Users\agnit\Documents\UT Austin\Fall'24\Adv Machine Learning\Project\hotel-id-2021-fgvc8\train_images"
train_dataset = HotelImageDataset(train_df, image_dir, Transform)
test_dataset = HotelImageDataset(test_df, image_dir, Transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH, shuffle=False)

# Load ResNet152 and remove the final classification layer
model = models.resnet152(pretrained=True)
model.fc = torch.nn.Identity()  # Replace the final layer with an identity layer
model = model.to(DEVICE)

# Extract features
def extract_features(loader):
    model.eval()
    features = []
    labels = []
    with torch.no_grad():
        for images, lbls in tqdm(loader, desc="Extracting Features", unit="batch"):
            images = images.to(DEVICE)
            feats = model(images)  # Get features from ResNet34
            features.append(feats.cpu())
            labels.extend(lbls.numpy())
    return torch.cat(features).numpy(), labels

train_features, train_labels = extract_features(train_loader)
test_features, test_labels = extract_features(test_loader)

# Train logistic regression
logreg = LogisticRegression(max_iter=1000)
logreg.fit(train_features, train_labels)

# Predict on test features
test_preds = logreg.predict(test_features)

# Evaluate
accuracy = accuracy_score(test_labels, test_preds)
print(f"Test Accuracy: {accuracy:.4f}")

# Save predictions
test_df['predicted_class_index'] = test_preds
test_df['predicted_hotel_id'] = test_df['predicted_class_index'].map(index_to_hotel_id)
test_df.to_csv("test_predictions_ResNet152_logreg.csv", index=False)


C:\Users\agnit\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\agnit\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to C:\Users\agnit/.cache\torch\hub\checkpoints\resnet152-394f9c45.pth
100%|███████████████████████████████████████████████████████████████████████████████| 230M/230M [00:15<00:00, 15.3MB/s]
Extracting Features: 100%|████████████████████████████████████████████████████████| 240/240 [11:09<00:00,

Test Accuracy: 0.4045


In [6]:
# Predict probabilities for test features
test_probs = logreg.predict_proba(test_features)

# Get top 3 predicted class indices for each test point
top3_preds = torch.topk(torch.tensor(test_probs), 3, dim=1).indices.numpy()

# Check if actual label is in the top 3 predictions
top3_correct = [
    test_labels[i] in top3_preds[i] for i in range(len(test_labels))
]

# Calculate top-3 accuracy
top3_accuracy = sum(top3_correct) / len(test_labels)
print(f"Top-3 Accuracy: {top3_accuracy:.4f}")


Top-3 Accuracy: 0.5267
